Download all the dependencies.

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

Download data file.

In [10]:
!wget -q -O 'edmonton_data.json' https://data.edmonton.ca/resource/jsc3-gmwb.json
print('Data downloaded!')

Data downloaded!


load the data.

In [11]:
with open('edmonton_data.json') as json_data:
    edmonton_data = json.load(json_data)

quick look

In [12]:
edmonton_data

[{'ward': 'WARD 1',
  'neighbourhood_number': '3140',
  'neighbourhood_name': 'CRESTWOOD',
  'english_only': '727',
  'arabic': '0',
  'cantonese': '11',
  'french': '18',
  'german': '7',
  'mandarin': '5',
  'north_american_indigenous': '0',
  'panjabi_punjabi': '2',
  'spanish': '2',
  'tagalog_pilipino_filipino': '14',
  'ukrainian': '5',
  'other': '67',
  'no_response': '58'},
 {'ward': 'WARD 1',
  'neighbourhood_number': '3330',
  'neighbourhood_name': 'PARKVIEW',
  'english_only': '570',
  'arabic': '0',
  'cantonese': '14',
  'french': '51',
  'german': '14',
  'mandarin': '3',
  'north_american_indigenous': '0',
  'panjabi_punjabi': '0',
  'spanish': '10',
  'tagalog_pilipino_filipino': '15',
  'ukrainian': '19',
  'other': '44',
  'no_response': '560'},
 {'ward': 'WARD 12',
  'neighbourhood_number': '6150',
  'neighbourhood_name': 'DALY GROVE',
  'english_only': '654',
  'arabic': '5',
  'cantonese': '11',
  'french': '14',
  'german': '7',
  'mandarin': '3',
  'north_americ

 look at the first item in this list.

In [13]:
df_data = edmonton_data
df_data[0]

{'ward': 'WARD 1',
 'neighbourhood_number': '3140',
 'neighbourhood_name': 'CRESTWOOD',
 'english_only': '727',
 'arabic': '0',
 'cantonese': '11',
 'french': '18',
 'german': '7',
 'mandarin': '5',
 'north_american_indigenous': '0',
 'panjabi_punjabi': '2',
 'spanish': '2',
 'tagalog_pilipino_filipino': '14',
 'ukrainian': '5',
 'other': '67',
 'no_response': '58'}

creating an empty dataframe.

In [14]:
# define the dataframe columns
column_names = ['Neighbourhood_Number', 'Borough', 'Neighborhood','Language'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

In [15]:
df

,Neighbourhood_Number,Borough,Neighborhood,Language


loop through the data and fill the dataframe.

In [16]:
for data in df_data:
    ward = data ['ward'] 
    neighbourhood_name = data ['neighbourhood_name']
    neighbourhood_number = data ['neighbourhood_number']
    panjabi_punjabi = data ['panjabi_punjabi']
    df = df.append({'Neighbourhood_Number': neighbourhood_number,
                                          'Borough': ward,
                                          'Neighborhood': neighbourhood_name,
                                          'Language': panjabi_punjabi}, ignore_index=True)

In [17]:
df.head()

,Neighbourhood_Number,Borough,Neighborhood,Language
0,3140,WARD 1,CRESTWOOD,2
1,3330,WARD 1,PARKVIEW,0
2,6150,WARD 12,DALY GROVE,98
3,4474,WARD 1,STARLING,1
4,5040,WARD 8,BELGRAVIA,4


In [18]:
df.shape

(388, 4)

Anather data File downloaded for geographical positioning.

In [19]:
df1 = pd.read_csv("https://data.edmonton.ca/api/views/3b6m-fezs/rows.csv?accessType=DOWNLOAD")
df1.head()

,Neighbourhood Number,Neighbourhood Name,Area Sq Km,Latitude,Longitude,Location
0,8886,Edmonton South East,20.424907,53.381222,-113.369114,"(53.38122208627588, -113.3691136163807)"
1,4670,Winterburn Industrial Area West,4.340207,53.562804,-113.701391,"(53.562803614238916, -113.70139052232909)"
2,4487,Secord,2.643762,53.533937,-113.701353,"(53.53393710914364, -113.7013527663504)"
3,4620,Westview Village,0.925300,53.552222,-113.696668,"(53.552222069483065, -113.69666819890175)"
4,6664,Decoteau North,3.750301,53.433217,-113.350674,"(53.43321724004238, -113.350674197389)"


In [20]:
df1.shape

(400, 6)

In [21]:
df2 = df1[['Neighbourhood Number', 'Latitude', 'Longitude']]
df2.head()

,Neighbourhood Number,Latitude,Longitude
0,8886,53.381222,-113.369114
1,4670,53.562804,-113.701391
2,4487,53.533937,-113.701353
3,4620,53.552222,-113.696668
4,6664,53.433217,-113.350674


In [22]:
df4 = df2.rename(columns={'Neighbourhood Number': 'Neighbourhood_Number'}, inplace=True)
df2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Neighbourhood_Number,Latitude,Longitude
0,8886,53.381222,-113.369114
1,4670,53.562804,-113.701391
2,4487,53.533937,-113.701353
3,4620,53.552222,-113.696668
4,6664,53.433217,-113.350674


In [23]:
df2.shape

(400, 3)

In [24]:
df.head()

,Neighbourhood_Number,Borough,Neighborhood,Language
0,3140,WARD 1,CRESTWOOD,2
1,3330,WARD 1,PARKVIEW,0
2,6150,WARD 12,DALY GROVE,98
3,4474,WARD 1,STARLING,1
4,5040,WARD 8,BELGRAVIA,4


In [25]:
df2.head()

,Neighbourhood_Number,Latitude,Longitude
0,8886,53.381222,-113.369114
1,4670,53.562804,-113.701391
2,4487,53.533937,-113.701353
3,4620,53.552222,-113.696668
4,6664,53.433217,-113.350674


Merge 2 data files to create single data files for assessment.

In [26]:
df['Neighbourhood_Number']=df['Neighbourhood_Number'].astype(int)
neighborhoods = pd.merge(df, df2, on='Neighbourhood_Number')
neighborhoods


,Neighbourhood_Number,Borough,Neighborhood,Language,Latitude,Longitude
0,3140,WARD 1,CRESTWOOD,2,53.535434,-113.569038
1,3330,WARD 1,PARKVIEW,0,53.524060,-113.567914
2,6150,WARD 12,DALY GROVE,98,53.449537,-113.400727
3,4474,WARD 1,STARLING,1,53.592264,-113.657843
4,5040,WARD 8,BELGRAVIA,4,53.511796,-113.532729
5,4670,WARD 1,WINTERBURN INDUSTRIAL AREA WEST,0,53.562804,-113.701391
6,6570,WARD 11,PARSONS INDUSTRIAL,0,53.459795,-113.479382
7,3050,WARD 2,BROWN INDUSTRIAL,0,53.582399,-113.568007
8,1210,WARD 6,RIVERDALE,2,53.544107,-113.469011
9,3320,WARD 2,OXFORD,25,53.622198,-113.547292


In [27]:
neighborhoods.shape

(382, 6)

In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 12 boroughs and 382 neighborhoods.


Dataframe with Bourough and language are seperated to find number of punjabi speaking household in given neighbourhoods.

In [85]:
df_iceland = neighborhoods[['Borough', 'Language',]]
df_iceland.head()

,Borough,Language
0,WARD 1,2
1,WARD 1,0
2,WARD 12,98
3,WARD 1,1
4,WARD 8,4


count of each ward in given data assessed.

In [93]:
df_Bouroughnumber = df_iceland.groupby('Borough').Borough.count()
df_Bouroughnumber

Borough
WARD 1     53
WARD 10    29
WARD 11    45
WARD 12    21
WARD 2     39
WARD 3     21
WARD 4     33
WARD 5     29
WARD 6     15
WARD 7     28
WARD 8     26
WARD 9     43
Name: Borough, dtype: int64

Punjabi speaking households for each ward is seperated by groupby code.

In [115]:
gb = df_iceland.groupby('Borough')
gb.sum()

,Language
Borough,
WARD 1,2010540305070010000130000000210042140010000000...
WARD 10,3013180276520348703040068124194922
WARD 11,0000000002401022616180250100513100800010012500...
WARD 12,9808417624278131291362683031474241804411941391...
WARD 2,02500119052712401000183130100205060060460000
WARD 3,130180206730507724358021076
WARD 4,261121248064066056048835148120003361245202680
WARD 5,97001501300621031701071104043230530
WARD 6,22020100212862013720


In [106]:
gb = df_iceland.groupby('Borough')
gb.count()

,Language
Borough,
WARD 1,53
WARD 10,29
WARD 11,45
WARD 12,21
WARD 2,39
WARD 3,21
WARD 4,33
WARD 5,29
WARD 6,15


#### Use geopy library to get the latitude and longitude values of edmonton

In [29]:
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Edmonton are 53.535411, -113.507996.


#### Create a map of edmonton with neighborhoods superimposed on top.

In [30]:
# create map of Edmonton using latitude and longitude values
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edmonton)  
    
map_edmonton

slice the original dataframe and create a new dataframe of the ward number12.

In [31]:
ward12_data = neighborhoods[neighborhoods['Borough'] == 'WARD 12'].reset_index(drop=True)
ward12_data.head()

,Neighbourhood_Number,Borough,Neighborhood,Language,Latitude,Longitude
0,6150,WARD 12,DALY GROVE,98,53.449537,-113.400727
1,6665,WARD 12,ANTHONY HENDAY SOUTHEAST,0,53.443016,-113.354281
2,6211,WARD 12,ELLERSLIE,84,53.428545,-113.457731
3,6580,WARD 12,POLLARD MEADOWS,176,53.449115,-113.413032
4,6530,WARD 12,MINCHAU,24,53.469829,-113.412766


Let's get the geographical coordinates of ward12.

In [33]:
address = 'WARD 12, EDMONTON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of WARD 12 are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of WARD 12 are 53.4742916, -113.3784859.


As we did with all of Edmonton City, let's visualizat WARD 12 the neighborhoods in it.

In [37]:
# create map of Manhattan using latitude and longitude values
map_ward12 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ward12_data['Latitude'], ward12_data['Longitude'], ward12_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ward12)  
    
map_ward12

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [38]:
CLIENT_ID = '3MC1YTB4GPNKECIDYDQEI5C0ENCMXRGJVGPB5PI4F125L2UX' # your Foursquare ID
CLIENT_SECRET = 'RJSKX3142TEYAU5SGPNCJ2WZYS1V53XKU5E1DBN0DDPFN52P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3MC1YTB4GPNKECIDYDQEI5C0ENCMXRGJVGPB5PI4F125L2UX
CLIENT_SECRET:RJSKX3142TEYAU5SGPNCJ2WZYS1V53XKU5E1DBN0DDPFN52P


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [39]:
ward12_data.loc[0, 'Neighborhood']

'DALY GROVE'

Get the neighborhood's latitude and longitude values.

In [40]:
neighborhood_latitude = ward12_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ward12_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = ward12_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of DALY GROVE are 53.449537, -113.400727.


#### Now, let's get the top 100 venues that are in Daly grove within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [41]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
 
radius = 500 # define radius
 
  # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=3MC1YTB4GPNKECIDYDQEI5C0ENCMXRGJVGPB5PI4F125L2UX&client_secret=RJSKX3142TEYAU5SGPNCJ2WZYS1V53XKU5E1DBN0DDPFN52P&v=20180605&ll=53.449537,-113.400727&radius=500&limit=100'

Send the GET request and examine the results

In [42]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de6c612bae9a2001cf9699e'},
 'response': {'headerLocation': 'Southeast Edmonton',
  'headerFullLocation': 'Southeast Edmonton, Edmonton',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 53.4540370045, 'lng': -113.39318481348714},
   'sw': {'lat': 53.4450369955, 'lng': -113.40826918651287}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57ef6c78498e317b7566c9fd',
       'name': 'Pandith Sri Sai Ram',
       'location': {'address': '2020 41 St NW',
        'lat': 53.452289,
        'lng': -113.404189,
        'labeledLatLngs': [{'label': 'display',
          'lat': 53.452289,
          'lng': -113.404189}],
        'distance': 382,
        'postalCode': 'T6L 6L2',
        'cc': '

let's borrow the **get_category_type** function from the Foursquare lab.

In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [44]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pandith Sri Sai Ram,Astrologer,53.452289,-113.404189
1,Shiva Parvathi ASTROLOGER & SPIRITUALIST,Astrologer,53.449122,-113.394981
2,Franklin's Inn Liquor Store,Liquor Store,53.449012,-113.393746
3,Junk Removal Company Edmonton,Auto Workshop,53.448002,-113.393658


And how many venues were returned by Foursquare?

In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in ward 12

#### Let's create a function to repeat the same process to all the neighborhoods in Ward 12.

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

####  code to run the above function on each neighborhood and create a new dataframe called *ward12_venues*.

In [47]:
ward12_venues = getNearbyVenues(names=ward12_data['Neighborhood'],
                                   latitudes=ward12_data['Latitude'],
                                   longitudes=ward12_data['Longitude']
                                  )

DALY GROVE
ANTHONY HENDAY SOUTHEAST
ELLERSLIE
POLLARD MEADOWS
MINCHAU
WALKER
CHARLESWORTH
THE ORCHARDS AT ELLERSLIE
JACKSON HEIGHTS
SILVER BERRY
TAMARACK
LARKSPUR
MAPLE
LAUREL
ELLERSLIE INDUSTRIAL
WILD ROSE
KINISKI GARDENS
BISSET
WEINLOS
SUMMERSIDE
CRAWFORD PLAINS


#### Let's check the size of the resulting dataframe

In [49]:
print(ward12_venues.shape)
ward12_venues

(36, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,DALY GROVE,53.449537,-113.400727,Pandith Sri Sai Ram,53.452289,-113.404189,Astrologer
1,DALY GROVE,53.449537,-113.400727,Shiva Parvathi ASTROLOGER & SPIRITUALIST,53.449122,-113.394981,Astrologer
2,DALY GROVE,53.449537,-113.400727,Franklin's Inn Liquor Store,53.449012,-113.393746,Liquor Store
3,DALY GROVE,53.449537,-113.400727,Junk Removal Company Edmonton,53.448002,-113.393658,Auto Workshop
4,ELLERSLIE,53.428545,-113.457731,Hooligan Welding Ltd.,53.430096,-113.455725,Shop & Service
5,ELLERSLIE,53.428545,-113.457731,Ellerslie Crossing Park,53.426099,-113.458920,Park
6,ELLERSLIE,53.428545,-113.457731,Grove Auto wrecking,53.428432,-113.452533,Business Service
7,ELLERSLIE,53.428545,-113.457731,Market at Summerside,53.424973,-113.462120,Shopping Plaza
8,POLLARD MEADOWS,53.449115,-113.413032,The Basement,53.452225,-113.409056,Bar
9,POLLARD MEADOWS,53.449115,-113.413032,Holy Family School Park,53.447041,-113.406453,Playground


Let's check how many venues were returned for each neighborhood

In [50]:
ward12_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BISSET,3,3,3,3,3,3
CHARLESWORTH,1,1,1,1,1,1
DALY GROVE,4,4,4,4,4,4
ELLERSLIE,4,4,4,4,4,4
ELLERSLIE INDUSTRIAL,1,1,1,1,1,1
JACKSON HEIGHTS,6,6,6,6,6,6
KINISKI GARDENS,2,2,2,2,2,2
LARKSPUR,3,3,3,3,3,3
MINCHAU,1,1,1,1,1,1


#### Let's find out how many unique categories can be curated from all the returned venues

In [51]:
print('There are {} uniques categories.'.format(len(ward12_venues['Venue Category'].unique())))

There are 28 uniques categories.


## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
ward12_onehot = pd.get_dummies(ward12_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ward12_onehot['Neighborhood'] = ward12_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ward12_onehot.columns[-1]] + list(ward12_onehot.columns[:-1])
ward12_onehot = ward12_onehot[fixed_columns]

ward12_onehot

dataframe size.

In [53]:
ward12_onehot.shape

(36, 29)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
ward12_grouped = ward12_onehot.groupby('Neighborhood').mean().reset_index()
ward12_grouped

,Neighborhood,Astrologer,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Electronics Store,Flower Shop,Home Service,Indian Restaurant,Kitchen Supply Store,Lawyer,Liquor Store,Park,Pet Store,Photography Studio,Pizza Place,Playground,Print Shop,Sandwich Place,Shop & Service,Shopping Mall,Shopping Plaza,Stables
0,BISSET,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.333333,0.333333,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
1,CHARLESWORTH,0.0,0.00,1.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
2,DALY GROVE,0.5,0.25,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
3,ELLERSLIE,0.0,0.00,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.250000,0.0,0.25,0.00
4,ELLERSLIE INDUSTRIAL,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
5,JACKSON HEIGHTS,0.0,0.00,0.0,0.0,0.00,0.166667,0.000000,0.166667,0.0,0.166667,0.000000,0.0,0.00,0.166667,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.166667,0.0,0.00,0.00
6,KINISKI GARDENS,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.5,0.00,0.00
7,LARKSPUR,0.0,0.00,0.0,0.0,0.00,0.000000,0.333333,0.000000,0.0,0.000000,0.333333,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
8,MINCHAU,0.0,0.00,0.0,1.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00
9,POLLARD MEADOWS,0.0,0.00,0.0,0.5,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.0,0.0,0.000000,0.0,0.00,0.00


confirm the new size

In [55]:
ward12_grouped.shape

(14, 29)

#### Let's print each neighborhood along with the top 5 most common venues

In [56]:
num_top_venues = 5

for hood in ward12_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ward12_grouped[ward12_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BISSET----
                venue  freq
0  Photography Studio  0.33
1                Park  0.33
2        Liquor Store  0.33
3          Astrologer  0.00
4       Auto Workshop  0.00


----CHARLESWORTH----
            venue  freq
0            Bank   1.0
1      Astrologer   0.0
2          Lawyer   0.0
3  Shopping Plaza   0.0
4   Shopping Mall   0.0


----DALY GROVE----
            venue  freq
0      Astrologer  0.50
1   Auto Workshop  0.25
2    Liquor Store  0.25
3  Shopping Plaza  0.00
4   Shopping Mall  0.00


----ELLERSLIE----
              venue  freq
0    Shopping Plaza  0.25
1  Business Service  0.25
2    Shop & Service  0.25
3              Park  0.25
4        Astrologer  0.00


----ELLERSLIE INDUSTRIAL----
                        venue  freq
0  Construction & Landscaping   1.0
1                  Astrologer   0.0
2                      Lawyer   0.0
3              Shopping Plaza   0.0
4               Shopping Mall   0.0


----JACKSON HEIGHTS----
                venue  freq
0      S

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ward12_grouped['Neighborhood']

for ind in np.arange(ward12_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ward12_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BISSET,Photography Studio,Park,Liquor Store,Stables,Flower Shop,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant
1,CHARLESWORTH,Bank,Stables,Shopping Plaza,Auto Workshop,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping
2,DALY GROVE,Astrologer,Auto Workshop,Liquor Store,Home Service,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
3,ELLERSLIE,Shop & Service,Business Service,Park,Shopping Plaza,Stables,Flower Shop,Auto Workshop,Bank,Bar,Chinese Restaurant
4,ELLERSLIE INDUSTRIAL,Construction & Landscaping,Stables,Shopping Plaza,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [61]:
# set number of clusters
kclusters = 5

ward12_grouped_clustering = ward12_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ward12_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 4, 0, 0, 0, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'value', kmeans.labels_)

ward12_merged = ward12_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ward12_merged = ward12_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ward12_merged.head() # check the last columns!

,Neighbourhood_Number,Borough,Neighborhood,Language,Latitude,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6150,WARD 12,DALY GROVE,98,53.449537,-113.400727,0.0,0.0,Astrologer,Auto Workshop,Liquor Store,Home Service,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
1,6665,WARD 12,ANTHONY HENDAY SOUTHEAST,0,53.443016,-113.354281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6211,WARD 12,ELLERSLIE,84,53.428545,-113.457731,0.0,0.0,Shop & Service,Business Service,Park,Shopping Plaza,Stables,Flower Shop,Auto Workshop,Bank,Bar,Chinese Restaurant
3,6580,WARD 12,POLLARD MEADOWS,176,53.449115,-113.413032,1.0,1.0,Bar,Playground,Stables,Home Service,Auto Workshop,Bank,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
4,6530,WARD 12,MINCHAU,24,53.469829,-113.412766,1.0,1.0,Bar,Stables,Shopping Plaza,Auto Workshop,Bank,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping


Finally, let's visualize the resulting clusters

In [71]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ward12_merged['Latitude'], ward12_merged['Longitude'], ward12_merged['Neighborhood'], ward12_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        
        fill=True,
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories

In [72]:
ward12_merged.loc[ward12_merged['Cluster Labels'] == 0, ward12_merged.columns[[1] + list(range(5, ward12_merged.shape[1]))]]

,Borough,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,WARD 12,-113.400727,0.0,0.0,Astrologer,Auto Workshop,Liquor Store,Home Service,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
2,WARD 12,-113.457731,0.0,0.0,Shop & Service,Business Service,Park,Shopping Plaza,Stables,Flower Shop,Auto Workshop,Bank,Bar,Chinese Restaurant
5,WARD 12,-113.430435,0.0,0.0,Stables,Home Service,Business Service,Kitchen Supply Store,Auto Workshop,Bank,Bar,Chinese Restaurant,Clothing Store,Coffee Shop
8,WARD 12,-113.410848,0.0,0.0,Indian Restaurant,Shop & Service,Pizza Place,Chinese Restaurant,Coffee Shop,Convenience Store,Home Service,Auto Workshop,Bank,Bar
9,WARD 12,-113.381406,0.0,0.0,Home Service,Sandwich Place,Stables,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
11,WARD 12,-113.381240,0.0,0.0,Clothing Store,Pet Store,Electronics Store,Stables,Home Service,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant
15,WARD 12,-113.381167,0.0,0.0,Print Shop,Park,Stables,Home Service,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store
16,WARD 12,-113.403303,0.0,0.0,Shopping Mall,Lawyer,Stables,Home Service,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store
17,WARD 12,-113.400659,0.0,0.0,Photography Studio,Park,Liquor Store,Stables,Flower Shop,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant


#### Cluster 2

In [73]:
ward12_merged.loc[ward12_merged['Cluster Labels'] == 1, ward12_merged.columns[[1] + list(range(5, ward12_merged.shape[1]))]]

,Borough,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,WARD 12,-113.413032,1.0,1.0,Bar,Playground,Stables,Home Service,Auto Workshop,Bank,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
4,WARD 12,-113.412766,1.0,1.0,Bar,Stables,Shopping Plaza,Auto Workshop,Bank,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping


#### Cluster 3

In [74]:
ward12_merged.loc[ward12_merged['Cluster Labels'] == 2, ward12_merged.columns[[1] + list(range(5, ward12_merged.shape[1]))]]

,Borough,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,WARD 12,-113.41813,2.0,2.0,Bank,Stables,Shopping Plaza,Auto Workshop,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping


#### Cluster 4

In [75]:
ward12_merged.loc[ward12_merged['Cluster Labels'] == 3, ward12_merged.columns[[1] + list(range(5, ward12_merged.shape[1]))]]

,Borough,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,WARD 12,-113.359165,3.0,3.0,Flower Shop,Stables,Shopping Plaza,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop


#### Cluster 5

In [76]:
ward12_merged.loc[ward12_merged['Cluster Labels'] == 4, ward12_merged.columns[[1] + list(range(5, ward12_merged.shape[1]))]]

,Borough,Longitude,value,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,WARD 12,-113.488153,4.0,4.0,Construction & Landscaping,Stables,Shopping Plaza,Auto Workshop,Bank,Bar,Business Service,Chinese Restaurant,Clothing Store,Coffee Shop
